## 概览
- 在这个案例中，我们将展示如何通过Paddle Quantum训练量子神经网络来将量子态进行对角化。

- 首先，让我们通过下面几行代码引入必要的library和package。

In [ ]:
import numpy
from numpy import diag
import scipy
from paddle import fluid
from paddle_quantum.circuit import UAnsatz
from paddle.complex import matmul, trace, transpose

$\newcommand{\ket}[1]{|{#1}\rangle}$
$\newcommand{\bra}[1]{\langle{#1}|}$

## 背景
量子态对角化算法（VQSD）[1-3] 目标是输出一个量子态的特征谱，即其所有特征值。求解量子态的特征值在量子计算中有着诸多应用，比如可以用于计算保真度和冯诺依曼熵，也可以用于主成分分析。
- 量子态通常是一个混合态，表示如下 $\rho_{\text{mixed}} = \sum_i P_i \ket{\psi_i}\bra{\psi_i}$
- 作为一个简单的例子，我们考虑一个2量子位的量子态，它的特征谱为 $(0.5, 0.3, 0.1, 0.1)$, 我们先通过随机作用一个酉矩阵来生成具有这样特征谱的量子态。


In [ ]:
scipy.random.seed(1)
V = scipy.stats.unitary_group.rvs(4) # 随机生成一个酉矩阵
D = diag([0.5, 0.3, 0.1, 0.1]) # 输入目标态rho的谱
V_H = V.conj().T 
rho = V @ D @ V_H # 生成 rho
print(rho) # 打印量子态 rho
rho = rho.astype('complex64')

## 搭建量子神经网络
- 在这个案例中，我们将通过训练量子神经网络QNN（也可以理解为参数化量子电路）来学习量子态的特征谱。这里，我们提供一个预设的2量子位量子电路。

- 我们预设一些该参数化电路的参数，比如宽度为2量子位。

- 初始化其中的变量参数，${\bf{\theta }}$代表我们量子神经网络中的参数组成的向量。
         

In [ ]:
N = 2 # 量子神经网络的宽度
SEED = 1 # 种子
THETA_SIZE = 14 # 网络中的参数

def U_theta(theta, N):
    """
    U_theta
    """

    cir = UAnsatz(N)
    cir.rz(theta[0], 1)
    cir.ry(theta[1], 1)
    cir.rz(theta[2], 1)

    cir.rz(theta[3], 2)
    cir.ry(theta[4], 2)
    cir.rz(theta[5], 2)

    cir.cnot([2, 1])

    cir.rz(theta[6], 1)
    cir.ry(theta[7], 2)

    cir.cnot([1, 2])

    cir.rz(theta[8], 1)
    cir.ry(theta[9], 1)
    cir.rz(theta[10], 1)

    cir.rz(theta[11], 2)
    cir.ry(theta[12], 2)
    cir.rz(theta[13], 2)

    return cir.state


## 配置训练模型 - 损失函数
- 现在我们已经有了数据和量子神经网络的架构，我们将进一步定义训练参数、模型和损失函数。
- 通过作用量子神经网络$U(\theta)$在量子态$\rho$后得到的量子态记为$\tilde\rho$，我们设定损失函数为$\tilde\rho$与用来标记的量子态$\sigma=0.1 \ket{00}\bra{00} + 0.2 \ket{01}\bra{01} + 0.3 \ket{10}\bra{10} + 0.4 \ket{11}\bra{11}$的内积。
- 具体的，设定损失函数为 $\mathcal{L}(\boldsymbol{\theta})  = Tr(\tilde\rho\sigma) .$

In [ ]:
sigma = diag([0.1, 0.2, 0.3, 0.4]) # 输入用来标记的量子态sigma
sigma = sigma.astype('complex64')

class Net(fluid.dygraph.Layer):
    """
    Construct the model net
    """

    def __init__(self, shape, rho, sigma, param_attr=fluid.initializer.Uniform(low=0.0, high=2 * numpy.pi, seed=SEED),
                 dtype='float32'):
        super(Net, self).__init__()

        self.rho = fluid.dygraph.to_variable(rho)
        self.sigma = fluid.dygraph.to_variable(sigma)

        self.theta = self.create_parameter(shape=shape, attr=param_attr, dtype=dtype, is_bias=False)

    def forward(self, N):
        """
        Args:
        Returns:
            The loss.
        """

        out_state = U_theta(self.theta, N)

        # rho_tilde 是将U_theta作用在rho后得到的量子态 
        rho_tilde = matmul(
            matmul(transpose(
                fluid.framework.ComplexVariable(out_state.real, -out_state.imag),
                perm=[1, 0]
            ), self.rho), out_state
        )

        # record the new loss
        loss = trace(matmul(self.sigma, rho_tilde))

        return loss.real, rho_tilde

## 配置训练模型 - 模型参数
在进行量子神经网络的训练之前，我们还需要进行一些训练（超）参数的设置，例如学习速率与迭代次数。
- 设定学习速率（learning rate）为0.1;
- 设定迭代次数为50次。

In [ ]:
ITR = 50 #训练的总的迭代次数

LR = 0.1 #学习速率

## 进行训练

- 当训练模型的各项参数都设置完成后，我们将数据转化为Paddle动态图中的变量，进而进行量子神经网络的训练。
- 过程中我们用的是Adam Optimizer，也可以调用Paddle中提供的其他优化器。
- 我们将训练过程中的结果依次输出。

In [ ]:
with fluid.dygraph.guard():
    # net
    net = Net(shape=[THETA_SIZE], rho=rho, sigma=sigma)

    # optimizer
    opt = fluid.optimizer.AdagradOptimizer(learning_rate=LR, parameter_list=net.parameters())
    # gradient descent loop
    for itr in range(ITR):
        loss, rho_tilde = net(N)

        rho_tilde_np = rho_tilde.numpy()
        loss.backward()
        opt.minimize(loss)
        net.clear_gradients()

        print('iter:', itr, 'loss:', '%.4f' % loss.numpy()[0])


## 总结
根据上面训练得到的结果，通过大概50次迭代，我们就比较好的完成了对角化。
我们可以通过打印$
\tilde{\rho} = U(\boldsymbol{\theta})\rho U^\dagger(\boldsymbol{\theta})
$
的来验证谱分解的效果。特别的，我们可以验证它的对角线与我们目标谱是非常接近的。

In [ ]:
print(rho_tilde_np)

print("The estimated spectrum is:", numpy.real(numpy.diag(rho_tilde_np)))
print("The target spectrum is:", numpy.diag(D))

### 参考文献

[1] R. Larose, A. Tikku, É. O. Neel-judy, L. Cincio, and P. J. Coles, “Variational quantum state diagonalization,” npj Quantum Inf., no. November 2018, 2019.

[2] K. M. Nakanishi, K. Mitarai, and K. Fujii, “Subspace-search variational quantum eigensolver for excited states,” Phys. Rev. Res., vol. 1, no. 3, p. 033062, Oct. 2019.

[3] M. Cerezo, K. Sharma, A. Arrasmith, and P. J. Coles, “Variational Quantum State Eigensolver,” arXiv:2004.01372, no. 1, pp. 1–14, Apr. 2020.
